In [ ]:
import argilla as rg
import sqlite3
import os
import numpy
import array
import json

In [ ]:
api_url = os.getenv("ARGILLA_API_URL", "http://localhost:6900")
api_key = os.getenv("ARGILLA_API_KEY", "argilla.apikey")

client = rg.Argilla(
    api_url=api_url,
    api_key=api_key
)

In [40]:
dataset = client.datasets("homefitness_dataset_v1")
if dataset is  None:
    settings = rg.Settings(
        guidelines="Classify the video  as fitness related or not.",
        fields=[
            rg.TextField(
                name="Title",
                title="Title",
                use_markdown=False,
            ),
            rg.TextField(
                name="Description",
                title="Description",
                use_markdown=False,
            ),
            rg.TextField(
                name="video_id",
                title="video_id",
                use_markdown=True,
            ),
            rg.TextField(
                name="channel",
                title="channeltitle",
                use_markdown=False,
            ),
             rg.TextField(
                name="view_count",
                title="view_count",
                use_markdown=False,
            ),
            
        ],
        questions=[
            rg.LabelQuestion(
                name="fit_label",
                title="Does this video belong to fitness or not?",
                labels=["Yes", "No"],
            )
        ],
        vectors=[rg.VectorField(
        name="title_desc_vector",
        title="Title Description Vector",
        dimensions=768
                )  ]
    )
    dataset = rg.Dataset(
        name=f"homefitness_dataset_v1",
        workspace="homeworkouts",
        settings=settings,
        client=client,
    )
    dataset.create()


/Users/kanantharaman/anaconda3/lib/python3.11/site-packages/argilla/client.py:338: UserWarning: Dataset with name 'homefitness_dataset_v1' not found in workspace 'homeworkouts'
  warnings.warn(f"Dataset with name {name!r} not found in workspace {workspace.name!r}")


In [ ]:
def dbconnection():
    # Connect to a local SQLite database file
    conn = sqlite3.connect('fitlabeler.db')
    return conn

In [ ]:
def getvideos(connection):
    cursor = connection.cursor()
    cursor.execute("""
        SELECT json_object(
            'video_id', videoid,
            'view_count', COALESCE(view_count, 0),
            'channel', c.channeltitle,
            'title', json_extract(v.payload, '$.title'),
            'description', COALESCE(json_extract(v.payload, '$.description'), ' '),
            'title_desc_vector', embedding
        )
        FROM yt_videos v
        JOIN yt_channels c ON v.channelid = c.channelid
        WHERE v.embedding IS NOT NULL
          AND v.status = 1
          AND json_extract(v.payload, '$.title') IS NOT NULL
        LIMIT 1000
    """)
    videos = cursor.fetchall()
    cursor.close()
    return videos


In [ ]:
def getsimilarvideos(connection, video_id):
    cursor = connection.cursor()
    # Get the embedding for the given video_id
    cursor.execute("SELECT embedding FROM yt_videos WHERE videoid = ?", (video_id,))
    row = cursor.fetchone()
    if row is None or row[0] is None:
        cursor.close()
        return []
    target_embedding = numpy.array(json.loads(row[0]))

    # Get candidate videos with embeddings
    cursor.execute("""
        SELECT videoid, COALESCE(view_count, 0), c.channeltitle,
               json_extract(v.payload, '$.title'),
               COALESCE(json_extract(v.payload, '$.description'), ' '),
               embedding
        FROM yt_videos v
        JOIN yt_channels c ON v.channelid = c.channelid
        WHERE v.embedding IS NOT NULL
          AND v.status = 1
          AND json_extract(v.payload, '$.title') IS NOT NULL
          AND v.videoid != ?
    """, (video_id,))
    videos = cursor.fetchall()

    # Compute distances
    results = []
    for vid in videos:
        emb = numpy.array(json.loads(vid[5]))
        dist = numpy.linalg.norm(target_embedding - emb)
        results.append((dist, vid))

    # Sort by distance and take top 10
    results.sort(key=lambda x: x[0])
    top_videos = results[:10]

    # Format as JSON-like tuples
    similar_videos = []
    for _, vid in top_videos:
        video_json = json.dumps({
            'video_id': vid[0],
            'view_count': vid[1],
            'channel': vid[2],
            'title': vid[3],
            'description': vid[4],
            'title_desc_vector': vid[5]
        })
        similar_videos.append((array.array('u', video_json),))  # mimic previous structure

    cursor.close()
    return similar_videos

In [ ]:
def updatevideo(connection, video_id):
    cursor = connection.cursor()
    cursor.execute("update yt_videos set status=2 where videoid=?", video_id=video_id)
    cursor.close()

In [64]:
def checkES(video_id):
    response = dataset.records(query=video_id).to_list(
    flatten=True
    )
    return response

In [66]:
if(checkES('Zh_lR10f2u8')):
    print('Record exists')

Record exists


In [ ]:
conn = dbconnection()
i=0
for video in getvideos(conn):
        record = []
        record.append(json.loads(video[0].read()))
        if(not checkES(record[0]['video_id'])):
            i=i+1
            dataset.records.log(record)
            updatevideo(conn,record[0]['video_id'])
        if(i%100==0):
            conn.commit()
conn.commit()
conn.close()
   

In [157]:
filtered_records = dataset.records(with_suggestions=True, with_responses=True).to_list(flatten=True)

In [158]:
rec_yes = []
rec_no = []
rec_none = []
for records in filtered_records:
     if( records['fit_label.responses'] is not None and records['fit_label.responses'][0] == 'Yes'):
         rec_yes.append(records)
     elif (records['fit_label.responses'] is not None and records['fit_label.responses'][0] == 'No'):
         rec_no.append(records)
     else:
         rec_none.append(records)

In [135]:
i=0
conn = dbconnection()
for records in filtered_records:
    if( records['fit_label.responses'] is not None and records['fit_label.responses'][0] == 'Yes'):
        continue
    i=i+1
    for video in getsimilarvideos(conn,records['video_id']):
        record = []
        record.append(json.loads(video[0].read()))
        if( records['fit_label.responses'] is not None and records['fit_label.responses'][0] == 'Yes'):
            record[0]['fit_label'] = 'Yes'
        else:
            record[0]['fit_label'] = 'No'
        if(not checkES(record[0]['video_id'])):
            dataset.records.log(record)
            updatevideo(conn,record[0]['video_id'])
    if(i%10==0):
        conn.commit()
conn.commit()
conn.close()

DatasetRecords: The provided batch size 256 was normalized. Using value 1.

Sending records...: 100%|██████████| 1/1 [00:00<00:00,  7.75batch/s]


DatasetRecords: The provided batch size 256 was normalized. Using value 1.

Sending records...: 100%|██████████| 1/1 [00:00<00:00,  4.59batch/s]


DatasetRecords: The provided batch size 256 was normalized. Using value 1.

...

Sending records...: 100%|██████████| 1/1 [00:00<00:00,  8.64batch/s]


DatasetRecords: The provided batch size 256 was normalized. Using value 1.

Sending records...: 100%|██████████| 1/1 [00:00<00:00,  7.24batch/s]


DatasetRecords: The provided batch size 256 was normalized. Using value 1.

Sending records...: 100%|██████████| 1/1 [00:00<00:00,  8.50batch/s]
